In [1]:
import os
import SimpleITK as sitk
import cv2


ModuleNotFoundError: No module named 'SimpleITK'

In [ ]:
def warp_label(moving_label_path, moving_img_path, fixed_img_path, fp):

    # gets the deformation field that warps the moving intensity image to
    # the fixed intensity image. The two images are usually the ED and ES images 
    # in a cardiac cycle. 

    fixedImage = sitk.ReadImage(fixed_img_path)
    movingImage = sitk.ReadImage(moving_img_path)
    elastixImageFilter = sitk.ElastixImageFilter()
    elastixImageFilter.SetFixedImage(fixedImage)
    elastixImageFilter.SetMovingImage(movingImage)
    # parameterMapVector = sitk.VectorOfParameterMap()
    # parameterMapVector.append(sitk.GetDefaultParameterMap("affine"))
    # parameterMapVector.append(sitk.GetDefaultParameterMap("bspline"))
    # elastixImageFilter.SetParameterMap(parameterMapVector)
    # elastixImageFilter.AddParameter( "Metric", "CorrespondingPointsEuclideanDistanceMetric" )
    parameterMapVector = sitk.VectorOfParameterMap()
    EulerTransform=sitk.GetDefaultParameterMap("rigid")
    EulerTransform["AutomaticScalesEstimation"]=( "true", )
    EulerTransform["AutomaticTransformInitialization"] = ( "true", ) 
    affine=sitk.GetDefaultParameterMap("affine")
    affine["AutomaticScalesEstimation"]=( "true", )
    affine["AutomaticTransformInitialization"] = ( "true", )
    bspline=sitk.GetDefaultParameterMap("bspline")
    bspline["AutomaticScalesEstimation"]=( "true", )
    bspline["AutomaticTransformInitialization"] = ( "true", ) 
    bspline['Metric0Weight']=(str(1),)
    spacing1=(747, 616)             # x and y dim of the fixed image
    spacing2=(747, 616)             # x and y dim of the moving image
    bspline['FinalGridSpacingInPhysicalUnits']=(str(max(spacing1)*6.),)
    fixedImage.SetSpacing(spacing1)
    movingImage.SetSpacing(spacing2)
    # bspline["Metric"]=(*bspline["Metric"],"DisplacementMagnitudePenalty")
    bspline["Metric"]=("AdvancedMeanSquares",bspline["Metric"][1])
    # bspline["Metric"] = ("TransformBendingEnergyPenalty", bspline["Metric"][1])
    # bspline["Metric"]=("AdvancedMeanSquares",bspline["Metric"][1])
    parameterMapVector.append(EulerTransform)
    parameterMapVector.append(bspline)
    elastixImageFilter.Execute()

    # resultImage = elastixImageFilter.GetResultImage()
    
    # The following steps warps the segmentation label corresponding to 
    # the moving image using the extracted deformation field. 
    transformParameterMap = elastixImageFilter.GetTransformParameterMap()
    transformixImageFilter = sitk.TransformixImageFilter()
    transformixImageFilter.SetTransformParameterMap(transformParameterMap)
    transformixImageFilter.SetMovingImage(sitk.ReadImage(moving_label_path)*30)
    transformixImageFilter.Execute()
    # sitk.WriteImage(transformixImageFilter.GetResultImage(), 'warped_'+ moving_label_path)

    sitk.WriteImage(sitk.Cast(sitk.RescaleIntensity(transformixImageFilter.GetResultImage()), sitk.sitkUInt8), f'warped/{fp}')

moving_dir = 'D:\\ICL\\Results\\CAMUS_EStoED_A2C\\ELASTIX\\val\\moving_img'
mask_dir = os.listdir(moving_dir)

for fp in mask_dir:
    rpos = fp.rfind('_', -24, -22)
    fixed_fp = fp[:rpos] + '_ED' + fp[rpos+3:]
    print(fixed_fp)
    
    fixed_img_path = f'val/fixed_img/{fixed_fp}'
    moving_img_path = f'val/moving_img/{fp}'
    moving_label_path = f'val/moving_msk/{fp}'
    
#     print(fixed_img_path)
#     print(moving_img_path)
    
    logo = sitk.ReadImage(fixed_img_path)
    
#     print(logo)
#     break

    warp_label(moving_label_path, moving_img_path, fixed_img_path, fp)
    
    break

# fixed_img_path = '/home/haobo/Desktop/val/fixed_img/patient0068_2CH_ES_CAMUS_EDtoES_A2C.png'
# moving_img_path = '/home/haobo/Desktop/val/moving_img/patient0068_2CH_ED_CAMUS_EDtoES_A2C.png'
# moving_label_path = '/home/haobo/Desktop/val/moving_msk/patient0068_2CH_ED_CAMUS_EDtoES_A2C.png'
# fp = 'patient0068_2CH_ED_CAMUS_EDtoES_A2C.png'
# warp_label(moving_label_path, moving_img_path, fixed_img_path, fp)

patient0008_2CH_ED_CAMUS_EStoED_A2C.png


AttributeError: module 'SimpleITK' has no attribute 'ElastixImageFilter'

In [ ]:
patient0008_2CH_ES_CAMUS_EStoED_A2C